# FCNN (Fully Convolutional Neural Network)

Este tipo de arquitecturas son conformadas por dos bloques principales, que conforman una trayectoria de codificador-decodificador o una trayectoria expansiva de contracción equivalente. En la figura se observan estas dos etapas y se muestra un ejemplo que cómo las características extraídas se hacen más particulares o especificas a medida que se profundiza en la red.

Los codificadores-decodificadores pueden definirse así:

* **Codificador (parte izquierda de la red)**: Codifica la imagen en una representación abstracta de las características de la imagen aplicando una secuencia de bloques convolucionales que disminuyen gradualmente la altura y la anchura de la representación, pero un número creciente de canales que corresponden a las características de la imagen.

* **Decodificador (parte derecha de la red)**: Decodifica la representación de la imagen en una máscara binaria aplicando una secuencia de convoluciones ascendentes (NO es lo mismo que la deconvolución) que aumenta gradualmente la altura y la anchura de la representación hasta el tamaño de la imagen original y disminuye el número de canales hasta el número de clases que estamos segmentando.


<div style="width:100%;text-align: center;">
<img src="https://i.imgur.com/WNCNVHS.png" width="800" height="400"/>
</div>

La FCNN es la arquitectura más utilizada para la segmentación de imágenes, entre ellas, se utiliza una en particular para imagenes médicas y se conoce como red U-net la cual tiene conexiones entre los dos bloques principales con el fin de aportar información a la etapa de reconstrucción del mapa de características.

# Objetivo de las redes U-Net
U-Net es una arquitectura de red convolucional para la segmentación rápida y precisa de imágenes. La intención de U-Net es captar tanto las características del contexto como las de la ubicación. La idea principal de la aplicación es utilizar capas de contracción sucesivas, a las que siguen inmediatamente operadores de remuestreo para obtener salidas de mayor resolución en las imágenes de entrada. 
# Arquitectura UNET
  

<div>
<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" width="900" height="800"/>
</div>


Al visualizar la forma de la arquitectura de la red, podemos ver por qué probablemente se llama U-Net. La arquitectura tiene forma de U, de ahí su nombre. La arquitectura de la red se ilustra en la figura anterior. Consta de una vía de contracción (lado izquierdo) y una vía de expansión (lado derecho) al igual que las FCNN y además contiene la conexión entre los dos bloque como se mencionó anteriormente. La ruta de contracción sigue la arquitectura típica de una red convolucional (CNN).



# Database para aplicar modelo U-net
La base de datos utilizada consiste en 800 imágenes de radiografía de tórax donde 394 imágenes corresponden a una manifestación de tuberculosis y 406 corresponden a radigrafías normales o de personas sanas. Esta base de datos es una combinación de dos bases de datos conocidas en el estado del arte para radiografía de tórax (Montgomery y Shenzen database).

## Etapas de pre-procesamiento y entrenamiento
1. Cargar la base de datos
2. Preparación de los datos
3. Imágenes de entrenamiento y de prueba
4. Construyendo una red U-net
5. Entrenamiento del modelo U-net
6. Métricas y validacón

# Configuración

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf # Libreria para aprendizaje automático
import os
import PIL # Librería de edición de imágenes
import re 

from skimage import segmentation
from glob import glob
from collections import defaultdict # Define un diccionario
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set(font_scale = 2)

from ipywidgets import interact

## 1. Cargar la base de datos 

Sobre los datos:
* Hay una pequeña anormalidad en la convención de nombres de las máscaras.
* Algunas imágenes no tienen sus correspondientes máscaras.

Comprobación de las radiografías y sus respectivas máscaras

In [ ]:
# Declarando los directorios de las imágenes
DIR = "/kaggle/input/chest-xray-masks-and-labels/data/"

lung_image_paths = glob(os.path.join(DIR,"Lung Segmentation/CXR_png/*.png")) # Directorio de las imagenes radiográficas.
mask_image_paths = glob(os.path.join(DIR,"Lung Segmentation/masks/*.png")) # Directorio de las máscaras, que indican la localización de pulmones.

related_paths = defaultdict(list)

# Combinando las imágenes de 1 para 1
for img_path in lung_image_paths:
    img_match = re.search("CXR_png/(.*)\.png$", img_path)
    if img_match:
        img_name = img_match.group(1)
    for mask_path in mask_image_paths:
        mask_match = re.search(img_name, mask_path)
        if mask_match:
            related_paths["image_path"].append(img_path)
            related_paths["mask_path"].append(mask_path)

paths_df = pd.DataFrame.from_dict(related_paths)
paths_df.head()

In [ ]:
Nimgs = paths_df.shape[0] # Número total de imagenes
@interact(xray_num = (0,Nimgs-1,1))
def plot_pair_img_mask(xray_num):
    img_path = paths_df["image_path"][xray_num]
    mask_path = paths_df["mask_path"][xray_num]

    img = PIL.Image.open(img_path)
    mask = PIL.Image.open(mask_path)
    
## Visualizar imagen e independientemente su máscara
#     fig = plt.figure(figsize = (10,10))

#     ax1 = fig.add_subplot(2,2,1)
#     ax1.imshow(img, cmap = "gray")
#     plt.title("Original");
#     plt.axis('off')
    
#     ax2 = fig.add_subplot(2,2,2)
#     plt.axis('off')
#     ax2.imshow(mask, cmap = "viridis")
#     plt.title("Máscara real");
#     fig.show()

    plt.figure(figsize = (5,5))
    plt.imshow(img, cmap = "gray")
    edges_est = segmentation.clear_border(np.squeeze(mask))
    plt.contour(edges_est,[0.5],colors=['red'])
    plt.axis('off')
    plt.show()

## 2. Preparación de los datos  
En este paso crearemos una función para tratar las imágenes de rayos X y las máscaras. Este proceso es necesario para poder preprocesar y normalizar las imágenes.  
Utilizaremos la biblioteca `cv2` para redimensionar las imágenes y las máscaras. 

In [ ]:
from tqdm import tqdm # librería para monitorear las funciones y sus tiempos de ejecución.
import cv2 # Librería Open-CV para python.

img_side_size = 256 # tamaño de las imágenes a procesar.

def prepare_train_test(df = pd.DataFrame(), resize_shape = tuple(), color_mode = "rgb"):
    img_array = list()
    mask_array = list()

    # Preparando Imagenes
    for image_path in tqdm(paths_df.image_path):
        resized_image = cv2.resize(cv2.imread(image_path),resize_shape) # Reescalando las imagenes a una resolución dada por el parámetro resize_shape
        resized_image = resized_image/255. # Normalizamos las intensidades de la imágen a valores entre 0 y 1. 
        # Procesar según el tipo de imagen (Color --> RGB | escala de grises --> Gray)
        if color_mode == "gray":
            img_array.append(resized_image[:,:,0])
        elif color_mode == "rgb":
            img_array.append(resized_image[:,:,:])
            
    # Preparando las mascaras
    for mask_path in tqdm(paths_df.mask_path):
        resized_mask = cv2.resize(cv2.imread(mask_path),resize_shape)
        resized_mask = resized_mask/255.
        mask_array.append(resized_mask[:,:,0])

    return img_array, mask_array

img_array, mask_array = prepare_train_test(df = paths_df, resize_shape = (img_side_size,img_side_size), color_mode = "gray")

## 3. Imágenes de entrenamiento y de prueba
Separación de los datos de entrenamiento y de prueba

In [ ]:
from sklearn.model_selection import train_test_split # Función para separar el conjunto de entrenamiento y de test.
img_train, img_test, mask_train, mask_test = train_test_split(img_array, mask_array, test_size = 0.2, random_state= 42)

# Se realiza un reshape para asegurar la forma del tensor adecuada: #img x W x H x C | donde W es el ancho de las imagenes, H es el alto y C el número de canales (gray C=1, RGB C=3)

img_side_size = 256
img_train = np.array(img_train).reshape(len(img_train), img_side_size, img_side_size, 1)
img_test = np.array(img_test).reshape(len(img_test), img_side_size, img_side_size, 1)
mask_train = np.array(mask_train).reshape(len(mask_train), img_side_size, img_side_size, 1)
mask_test = np.array(mask_test).reshape(len(mask_test), img_side_size, img_side_size, 1)

## 4. Construyendo una red U-net

* Las métricas utilizadas para evaluar los resultados son el coeficiente Dice e IoU (Intersection over Union):

<div style="width:100%;text-align: center;">
<img src="https://www.researchgate.net/publication/328671987/figure/fig4/AS:688210103529478@1541093483784/Calculation-of-the-Dice-similarity-coefficient-The-deformed-contour-of-the-liver-from.ppm" width="500" height="300"/>
</div>

<div style="width:100%;text-align: center;">
<img src="https://i.imgur.com/yJp0n0n.png" width="500" height="300"/>
</div>


In [ ]:
# Función que calcula la medida Dice 
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

# Función que calcula la pérdida según el Dice.
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Función Dice para test
def dice_coef_test(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    if union==0: return 1
    intersection = np.sum(y_true_f * y_pred_f)
    return 2. * intersection / union

# Intersection over Union metric
def IOU(y_true,y_pred):
    intersection = np.sum(y_true * y_pred)
    sum_ = np.sum(y_true) + np.sum(y_pred)
    jac = (intersection + 1) / (sum_ - intersection + 1)
    return jac

* Conv2D: Para crear las capas de convolución 2D, utilizaremos los siguientes filtros = [32, 64, 256 y 512]. Esta capa crea un núcleo de convolución que ayuda a producir un tensor de salida.

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.activations import *
from tensorflow.keras.optimizers import Adam


def unet(input_size=(img_side_size,img_side_size,1)):
    inputs = Input(input_size)
    """
    codifica la imagen en una representación abstracta de las características de la imagen aplicando 
    una secuencia de bloques convolucionales que disminuyen gradualmente la altura y la anchura de la representación
    """
    # codificador (parte izquierda de la “U”)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    """
    decodifica la representación de la imagen en una máscara binaria aplicando una secuencia de convoluciones ascendentes.
    que aumenta gradualmente la altura y la anchura de la representación hasta el tamaño de la imagen original y disminuye el número de 
    de canales al número de clases que estamos segmentando
    """
    # decodificador (parte derecha de la “U”)
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    
    #
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

In [ ]:
EPOCHS = 10
input_size=(img_side_size,img_side_size,1)

model = unet(input_size=(img_side_size,img_side_size,1)) # Creamos el modelo 
model.compile( # Opciones de compilación
    optimizer=Adam(learning_rate=5*1e-4), 
    loss="binary_crossentropy",
    metrics=[dice_coef, 'binary_accuracy']
)

model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,
           to_file="model.png",
           show_shapes=True,
           show_layer_names=True)

## 5. Entrenamiento del modelo U-Net

Para nuestro entrenamiento utilizaremos `epochs = 30`, que es el número de veces que recorreremos el conjunto de lotes de entrenamiento.

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    patience=10
) # Callback, si después de 10 epocas, la pérdida no se reduce o cambia. Se para el entrenamiento de forma temprana.

history = model.fit(
    x = img_train, 
    y = mask_train, 
    validation_data = (img_test, mask_test), 
    epochs = 30,
    batch_size = 16,
    callbacks = [earlystopping]
)

## Guardado del modelo en la salida

In [ ]:
model.save('./model_seg.h5')

### Visualización del resultado
Aquí crearemos una función para visualizar la imagen de vista previa, la imagen de la máscara real, la superposición sobre la máscara real y la imagen original.

In [ ]:
Nimgs = img_test.shape[0] # Número total de imagenes
img_array = img_test
@interact(img_num = (0,Nimgs-1,1))
def test_on_image( img_num ):
    """ para datos de previsión >= .5 asignaremos 1, en caso contrario el valor será 0 # 6. métricas y validación  
**Entropía cruzada (pérdida)**: para cuantificar la diferencia entre las dos distribuciones de probabilidad (entrenamiento y validación).
* **Coeficiente de los dados**: El coeficiente de Dice es un estadístico utilizado para medir la similitud de dos muestras, una de las métricas más utilizadas en el contexto de la segmentación de imágenes."""
    pred = model.predict(img_array[img_num].reshape(1,img_side_size,img_side_size,1))
    pred[pred>0.5] = 1.0
    pred[pred<0.5] = 0.0
    
    dice = dice_coef_test(y_true = mask_test[img_num], y_pred = pred)
    iou = IOU(y_true = mask_test[img_num], y_pred = pred)
    fig = plt.figure(figsize = (15,10))
    
    plt.subplot(1,3,1)
    plt.imshow(pred.reshape(img_side_size, img_side_size), cmap = "PuBu")
    plt.title("Predicción")
    plt.axis("off")
    
    plt.subplot(1,3,2)
    plt.imshow(mask_test[img_num].reshape(img_side_size, img_side_size), cmap = "viridis")
    plt.title("Máscara real");
    plt.axis("off")
    
    plt.subplot(1,3,3)
#     plt.figure(figsize = (5,5))
    plt.imshow(img_array[img_num].reshape(img_side_size, img_side_size), cmap = "gray")
    edges_est1 = segmentation.clear_border(np.squeeze(mask_test[img_num].reshape(img_side_size, img_side_size)))
    edges_est2 = segmentation.clear_border(np.squeeze(pred.reshape(img_side_size, img_side_size)))
    plt.contour(edges_est1,[0.5],colors=['red'])
    plt.contour(edges_est2,[0.5],colors=['blue'])
    
    plt.title('Superposición')
    plt.axis('off')
#     plt.savefig('./prediction.png', bbox_inches='tight', pad_inches=0)
    plt.suptitle('Dice: {:2.2%} and IOU: {:2.2%}'.format(dice,iou))
    plt.show()
    
    
#     plt.imshow(img_array[img_num].reshape(img_side_size, img_side_size), cmap = "gray")
#     plt.imshow(mask_test[img_num].reshape(img_side_size, img_side_size), cmap = "viridis", alpha = 0.5)
#     plt.imshow(pred.reshape(img_side_size, img_side_size),cmap = "PuBu", alpha = 0.3)
#     plt.title("Superposición")
#     plt.axis("off")
    
#     plt.subplot(1,4,4)
#     plt.imshow(img_array[img_num].reshape(img_side_size, img_side_size), cmap = "gray")
#     plt.title("Original")
#     plt.axis("off")
    
    return None


## 6. Métricas y validación  
+ **Cross-entropy (pérdida)**: para cuantificar la diferencia entre las dos distribuciones de probabilidad (entrenamiento y validación).
* **Coeficiente Dice**: El coeficiente de Dice es un estadístico utilizado para medir la similitud de dos muestras, una de las métricas más utilizadas en el contexto de la segmentación de imágenes.

In [ ]:
def get_metrics(history):
    fig = plt.figure(figsize = (15,15))
    plt.subplot(2,2,1)
    plt.plot(history.history["loss"], label = "loss entrenamiento")
    plt.plot(history.history["val_loss"], label = "loss validación")
    plt.title("Validación x Entrenamiento: Entropía binaria", fontsize=18, y=1)
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Entropia binaria")

    plt.subplot(2,2,2)
    plt.plot(history.history["dice_coef"], label = "Coeficiente Dice entrenameinto")
    plt.plot(history.history["val_dice_coef"], label = "Coeficiente Dice validación")
    plt.title("Validación x Entrenamiento: Coeficiente Dice", fontsize=18, y=1)
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Coeficiente Dice")
    
get_metrics(history = history)

### Comprobación de la validación de la prueba mediante el coeficiente Dice  
Utilizaremos los datos de la prueba para hacer nuestra predicción y realizaremos el solapamiento entre la máscara predicha y la real. Se examinarán 141 imágenes.

In [ ]:
dice_coefs = list()
pred = model.predict(img_test)
pred[pred>=0.5] = 1
pred[pred<0.5] = 0
for i in tqdm(range(len(img_test))):
    prediction = pred[i]
    dice = dice_coef_test(y_true = mask_test[i], y_pred = prediction)
    dice_coefs.append(dice)

In [ ]:
fig, ax = plt.subplots(figsize =(10, 7))
plt.hist(np.array(dice_coefs), bins = 50)
plt.title("Distribución del Coeficiente Dice (141 imágenes)",fontsize=20, y=1)
plt.xlabel("Coeficiente Dice");
plt.ylabel("Número de imágenes");
print(f'Mediana Coef. Dice: {np.median(np.array(dice_coefs))}');
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize =(10, 7))
plt.boxplot(np.array(dice_coefs))
plt.title("Distribución del Coeficiente Dice (141 imágenes)",fontsize=20, y=1)
plt.ylabel("Coeficiente Dice");
print(f'Mediana Coef. Dice: {np.median(np.array(dice_coefs))}');
plt.show()

* ¿Qué sucede si la pérdida no es la Cross-entropy, sino el coeficiente Dice?
* ¿Lo anterior es posible?